In [ ]:
!qsmxt bids out \
    --premade 'body' \
    --do_qsm \
    --do_t2starmap \
    --do_r2starmap \
    --do_swi \
    --mask_erosions 3 0 \
    --threshold_value 25.0 \
    --slurm a_barth general \
    --auto_yes

In [1]:
import os
import glob
import nibabel as nib
import numpy as np
import enum
import json
import SimpleITK as sitk
from scipy.ndimage import label
from scipy.ndimage import binary_dilation, binary_erosion, generate_binary_structure, label, center_of_mass
import pandas as pd

/home/uqaste15/miniconda3/envs/prostate39/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/uqaste15/miniconda3/envs/prostate39/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [113]:
# --- Data preparation for bids-2024 and bids-2025 ---
# The script collects subjects from both datasets and combines the modality lists.
bids_dirs = ["bids-2024", "bids-2025"]

# Setup dictionary to hold paths for each subject
# This will be used to create a pandas dataframe later
# Initialize an empty list to hold the paths
paths_list = []

for bids_dir in bids_dirs:
    subject_dirs = sorted(glob.glob(os.path.join(bids_dir, "sub-*")))
    subject_dirs = [s for s in subject_dirs if 'sub-z0449294' not in s]

    for subject_dir in subject_dirs:
        subject_id = os.path.basename(subject_dir)

        # Get all nifti files
        nii_paths = glob.glob(os.path.join(subject_dir, "ses-*", "*", "*.nii"))

        paths_dict = {}

        for nii_path in nii_paths:
            # Extract the modality name from the file name
            modality_name = os.path.basename(nii_path).split('.')[0]
            if modality_name.startswith('sub-'):
                continue
            paths_dict[modality_name] = nii_path

        # Append the dictionary to the list
        paths_list.append(paths_dict)
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(paths_list)
# Replace NaN values with None
df = df.where(pd.notnull(df), None)

df

,qsm_siemens_resliced,t2,t1,t1_corrected_segmentation_clean,qsm_siemens_segmentation_clean,b0,qsm_siemens,qsm_siemens_segmentation,t1_corrected,magnitude,ct_space-t1,ct,t1_segmentation,qsm_qsmxt,r2s,t2s,swi,swi_mip,ct_roi,t1_corrected_t1_segmentation_clean
0,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,bids-2024/sub-z0002292/ses-20240318/extra_data...,None,None,None,None,None,None,None
1,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,bids-2024/sub-z0163277/ses-20240123/extra_data...,None,None
2,bids-2024/sub-z0206923/ses-20231003/extra_data...,None,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,bids-2024/sub-z0206923/ses-20231003/extra_data...,None
3,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,bids-2024/sub-z0232771/ses-20240228/extra_data...,None,None
4,bids-2024/sub-z0347933/ses-20240129/extra_data...,None,bids-2024/sub-z0347933/ses-20240129/extra_data...,bids-2024/sub-z0347933/ses-20240129/extra_data...,bids-2024/sub-z0347933/ses-20240129/extra_data...,bids-2024/sub-z0347933/ses-20240129/extra_data...,bids-2024/sub-z0347933/ses-20240129/extra_data...,bids-2024/sub

In [ ]:

def apply_homogeneity_correction(image_data):
    # Convert the NumPy array to a SimpleITK image.
    sitk_image = sitk.GetImageFromArray(image_data)
    # Create a mask using Otsu thresholding.
    mask = sitk.OtsuThreshold(sitk_image, 0, 1, 200)
    # Run the N4 bias field correction.
    corrected_image = sitk.N4BiasFieldCorrection(sitk_image, mask)
    # Convert the corrected SimpleITK image back to a NumPy array.
    corrected_data = sitk.GetArrayFromImage(corrected_image)
    return corrected_data

def append_suffix_to_filename(filename, suffix):
    # Handle files ending with '.nii.gz'
    if filename.endswith('.nii.gz'):
        base = filename[:-7]
        return base + suffix + '.nii.gz'
    else:
        base, ext = os.path.splitext(filename)
        return base + suffix + ext

# Process each T1w image.
for t1_file in t1_files:
    new_file = append_suffix_to_filename(t1_file, '_homogeneity-corrected')
    
    if os.path.exists(new_file):
        print(f"Skipping {t1_file} because {new_file} already exists.")
        continue

    nii = nib.load(t1_file)
    data = nii.get_fdata()
    corrected_data = apply_homogeneity_correction(data)
    
    corrected_img = nib.Nifti1Image(corrected_data, nii.affine, nii.header)
    nib.save(corrected_img, new_file)
    print(f"Saved corrected image to {new_file}")

In [116]:
from nilearn.image import resample_to_img

def resample_image_to_target(source_nii, target_nii, interpolation='continuous'):
        resampled_source = resample_to_img(
            source_nii,
            target_nii,
            interpolation=interpolation,
            force_resample=True,
            copy_header=True
        )

        # Acquire the spacing (zoom values) from the 
        target_spacing = target_nii.header.get_zooms()[:3]

        # Get image data and force the affine and zooms to match the target
        resampled_data = resampled_source.get_fdata()
        new_img = nib.Nifti1Image(resampled_data, target_nii.affine, target_nii.header)
        new_img.header.set_zooms(target_spacing)
        
        return new_img

# turn the above into a function
def resample_all_images(df, source_col, target_col, new_filename, interpolation='nearest'):
    for index, row in df.iterrows():
        source_nii = row[source_col]
        target_nii = row[target_col]

        if source_nii and target_nii:
            # Load the images
            source_img = nib.load(source_nii)
            target_img = nib.load(target_nii)

            # Resample the source image to the target image
            source_resampled = resample_image_to_target(source_img, target_img, interpolation)

            # Copy the header and affine from the target image
            source_resampled = nib.Nifti1Image(
                source_resampled.get_fdata(),
                target_img.affine,
                target_img.header
            )

            # Save the resampled image
            out_filepath = os.path.join(os.path.dirname(source_nii), new_filename)
            nib.save(source_resampled, out_filepath)
            print(f"Saved resampled image to {out_filepath}")

In [117]:
# Resample CT segmentations to QSM
resample_all_images(
    df=df,
    source_col='t1_segmentation',
    target_col='qsm_siemens',
    new_filename='qsm_siemens_segmentation.nii',
    interpolation='nearest'
)

Saved resampled image to bids-2024/sub-z0002292/ses-20240318/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0163277/ses-20240123/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0206923/ses-20231003/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0232771/ses-20240228/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0347933/ses-20240129/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0381949/ses-20231005/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0742379/ses-20240124/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z0747844/ses-20240304/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z1318033/ses-20230825/extra_data/qsm_siemens_segmentation.nii
Saved resampled image to bids-2024/sub-z1384048/ses-20231108/extra_data/qsm_siemens_segment

# Clean up segmentations

In [118]:
import enum
import numpy as np
import nibabel as nib
import os
from scipy.ndimage import (
    label,
    binary_dilation,
    generate_binary_structure,
    center_of_mass
)
import pandas as pd

# Define segmentation types
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

def create_spherical_mask(center, shape, radius):
    indices = np.indices(shape).transpose((1, 2, 3, 0))
    distances = np.linalg.norm(indices - center, axis=-1)
    return distances <= radius

def enforce_2d_single_component(mask):
    structure2d = generate_binary_structure(2, 1)  # 4-connected in 2D
    new_mask = mask.copy()
    for axis in range(3):
        for idx in range(mask.shape[axis]):
            if axis == 0:
                slice_mask = new_mask[idx, :, :]
            elif axis == 1:
                slice_mask = new_mask[:, idx, :]
            elif axis == 2:
                slice_mask = new_mask[:, :, idx]
            labeled_slice, num_features = label(slice_mask, structure=structure2d)
            if num_features > 1:
                sizes = [np.sum(labeled_slice == comp_id) for comp_id in range(1, num_features + 1)]
                largest_comp = np.argmax(sizes) + 1
                new_slice = (labeled_slice == largest_comp)
                if axis == 0:
                    new_mask[idx, :, :] = new_slice
                elif axis == 1:
                    new_mask[:, idx, :] = new_slice
                elif axis == 2:
                    new_mask[:, :, idx] = new_slice
    return new_mask

def gaussian_filter3d(input_data, sigma):
    from scipy.ndimage import gaussian_filter
    return gaussian_filter(input_data, sigma=sigma)

def clean_segmentation(seg_nii, input_nii, stdevs=1, calc_stdevs=1.2):
    # Load the segmentation and input image data
    seg = np.array(np.round(seg_nii.get_fdata()), dtype=np.uint8)
    input_data = input_nii.get_fdata()

    seed_mask_labelled, num_labels = label(
        input=(seg == SegType.GOLD_SEED.value),
        structure=np.ones((3, 3, 3))
    )
    seed_mask = np.zeros(seg.shape, dtype=bool)

    # Get prostate values
    prostate_values = input_data[seg == SegType.PROSTATE.value]

    for seed_id in range(1, num_labels + 1):
        seed_i_mask = (seed_mask_labelled == seed_id)
        coords = np.column_stack(np.where(seed_i_mask))
        min_voxel = tuple(coords[np.argmin(input_data[seed_i_mask])])
        centre_voxel = center_of_mass(seed_i_mask)
        middle_voxel = np.array([
            int((min_voxel[0] + centre_voxel[0]) / 2),
            int((min_voxel[1] + centre_voxel[1]) / 2),
            int((min_voxel[2] + centre_voxel[2]) / 2)
        ])
        seed_i_mask = create_spherical_mask(middle_voxel, input_data.shape, radius=4)
        
        seed_i_mask = seed_i_mask & (input_data < (prostate_values.mean() - stdevs * np.std(prostate_values)))

        # create a 3D T structure that avoids diagonal connections
        structure = np.array([
        [
            [0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]
        ],[ 
            [0, 1, 0],
            [1, 1, 1],
            [0, 1, 0]
        ],[
            [0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]
        ]], dtype=bool)

        seed_i_mask_labelled, num_labels_comp = label(seed_i_mask, structure=structure)
        if num_labels_comp > 1:
            sizes = [np.sum(seed_i_mask_labelled == label_id) for label_id in range(1, num_labels_comp + 1)]
            largest_label = np.argmax(sizes) + 1
            seed_i_mask = (seed_i_mask_labelled == largest_label)

        max_iters = 2
        iter_idx = 0
        current_stdevs = stdevs
        accum_mask = seed_i_mask.copy()

        while iter_idx < max_iters:
            candidate = binary_dilation(accum_mask)
            candidate_new = candidate & (input_data < (np.mean(prostate_values) - current_stdevs * np.std(prostate_values))) & (~accum_mask)
            if not candidate_new.any():
                break
            new_mask = accum_mask | candidate_new
            new_mask = enforce_2d_single_component(new_mask)
            if np.array_equal(new_mask, accum_mask):
                break
            accum_mask = new_mask
            iter_idx += 1
            current_stdevs += 1
        seed_i_mask = accum_mask
        seed_mask = np.logical_or(seed_mask, seed_i_mask)

    seed_mask_indices = np.argwhere(seed_mask)
    if seed_mask_indices.size == 0:
        seed_mask_center = np.array([input_data.shape[0] // 2, input_data.shape[1] // 2, input_data.shape[2] // 2])
        max_distance = 0
    else:
        min_indices = np.min(seed_mask_indices, axis=0)
        max_indices = np.max(seed_mask_indices, axis=0)
        seed_mask_center = np.array([
            int((min_indices[0] + max_indices[0]) / 2),
            int((min_indices[1] + max_indices[1]) / 2),
            int((min_indices[2] + max_indices[2]) / 2)
        ])
        max_distance = 0
        seed_mask_labelled, num_seed_labels = label(seed_mask, structure=np.ones((3, 3, 3)))
        for seed_id in range(1, num_seed_labels + 1):
            cur_seed_mask = (seed_mask_labelled == seed_id)
            cur_center = center_of_mass(cur_seed_mask)
            distance = np.linalg.norm(seed_mask_center - cur_center)
            if distance > max_distance:
                max_distance = distance

    calc_mask = input_data < (prostate_values.mean() - calc_stdevs * np.std(prostate_values))
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)

    calc_mask = binary_dilation(calc_mask, iterations=1)
    calc_mask = gaussian_filter3d(calc_mask.astype(np.float32), sigma=calc_stdevs)
    calc_mask = calc_mask > 0.5

    calc_mask_labelled, num_calc = label(calc_mask, structure=np.ones((3, 3, 3)))
    for calc_id in range(1, num_calc + 1):
        if np.sum(calc_mask_labelled == calc_id) == 1:
            calc_mask[calc_mask_labelled == calc_id] = False

    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0
    seg[calc_mask] = 2
    seg[seed_mask] = 1

    seg_clean_nii = nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine)
    return seg_clean_nii

# turn the above into a function
def clean_all_segmentations(df, input_col, seg_col, new_filename):
    for index, row in df.iterrows():
        input_path = row[input_col]
        seg_path = row[seg_col]

        if input_path and seg_path:
            # Load the images
            input_img = nib.load(input_path)
            seg_img = nib.load(seg_path)

            # Clean the segmentation
            cleaned_segmentation = clean_segmentation(seg_img, input_img)

            # Save the cleaned segmentation
            out_filepath = os.path.join(os.path.dirname(input_path), new_filename)
            nib.save(cleaned_segmentation, out_filepath)
            print(f"Saved cleaned segmentation to {out_filepath}")

In [119]:
# Use the above function instead
clean_all_segmentations(
    df=df,
    input_col='qsm_siemens',
    seg_col='qsm_siemens_segmentation',
    new_filename='qsm_siemens_segmentation_clean.nii'
)

Saved cleaned segmentation to bids-2024/sub-z0002292/ses-20240318/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0163277/ses-20240123/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0206923/ses-20231003/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0232771/ses-20240228/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0347933/ses-20240129/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0381949/ses-20231005/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0742379/ses-20240124/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z0747844/ses-20240304/extra_data/qsm_siemens_segmentation_clean.nii
Saved cleaned segmentation to bids-2024/sub-z1318033/ses-20230825/extra_data/qsm_siemens_segmentation_cl

# Combine magnitude images

In [ ]:
for session_dir in session_dirs:
    mag_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*run-01*part-mag*nii*")))
    nii = nib.load(mag_images[0])
    mag_4d = np.array([nib.load(mag_images[i]).get_fdata() for i in range(len(mag_images))])
    mag_combined = np.sqrt(np.sum(np.square(mag_4d), axis=0))
    filename = os.path.join(session_dir, "extra_data", "magnitude_combined.nii")
    nib.save(nib.Nifti1Image(mag_combined, header=nii.header, affine=nii.affine), filename)
    print(f"Saved {filename}")

# Generate DICOMs for Radiographers

In [ ]:
for session_dir in session_dirs[:4]:
    subject_id = session_dir.split(os.sep)[1]
    
    B0_file = [B0_file for B0_file in fmap_files if subject_id in B0_file][0]
    qsm_file = [qsm_file for qsm_file in qsm_files if subject_id in qsm_file][0]
    mag_file = [mag_file for mag_file in mag_files if subject_id in mag_file][0]
    
    !mkdir -p {os.path.join(session_dir, "extra_data", "B0_dcm")}
    !mkdir -p {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")}
    !mkdir -p {os.path.join(session_dir, "extra_data", "qsm_dcm")}
    
    !nii2dcm --dicom_type MR --centered {B0_file} {os.path.join(session_dir, "extra_data", "B0_dcm")}
    !nii2dcm --dicom_type MR {mag_file} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")}
    !nii2dcm --dicom_type MR --centered {qsm_file} {os.path.join(session_dir, "extra_data", "qsm_dcm")}

    !tar cf {subject_id}.tar {os.path.join(session_dir, "extra_data", "B0_dcm")} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")} {os.path.join(session_dir, "extra_data", "qsm_dcm")}

    !rm -rf {os.path.join(session_dir, "extra_data", "B0_dcm")} {os.path.join(session_dir, "extra_data", "magnitude_combined_dcm")} {os.path.join(session_dir, "extra_data", "qsm_dcm")}